In [48]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
import pytrends
import matplotlib.pyplot as plt

In [50]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

kw_list=["Bitcoin crashing"]
data = pytrends.get_historical_interest(kw_list, year_start=2018, month_start=1, day_start=1, hour_start=0, year_end=2018, month_end=2, day_end=2, hour_end=0, cat=0, geo='', gprop='', sleep=0)

In [51]:
data

,Bitcoin crashing,isPartial
date,,
2018-01-01 00:00:00,22,False
2018-01-01 01:00:00,47,False
2018-01-01 02:00:00,75,False
2018-01-01 03:00:00,78,False
2018-01-01 04:00:00,52,False
...,...,...
2018-02-01 20:00:00,77,False
2018-02-01 21:00:00,23,False
2018-02-01 22:00:00,28,False


In [65]:
kw_list=["how to buy bitcoin", "bitcoin crash", "where to buy bitcoin", "sell bitcoin", "bitcoin price"]
data2 = pytrends.get_historical_interest(kw_list, year_start=2017, month_start=6, day_start=1, hour_start=0, year_end=2018, month_end=2, day_end=2, hour_end=0, cat=0, geo='', gprop='', sleep=60)

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConne

In [61]:
len(data2)
# data2.iloc[::(24*2), :].plot()
# data.plot()

0

In [26]:
plt.plot(np.asarray(data["date"], dtype='datetime64[s]'),data["Bitcoin crashing"])

KeyError: 'date'

In [187]:
r = requests.get('https://api.pro.coinbase.com/products/BTC-USD/candles?start=2020-12-27T00:00:00Z&end=2020-12-28T00:00:00Z&granularity=3600')
r.json()

[[1609113600, 26086.19, 26955.44, 26255.16, 26845.64, 1418.01702094],
 [1609110000, 26205.11, 26677.72, 26493.96, 26258.65, 1276.38185369],
 [1609106400, 26255, 26637.69, 26390.97, 26493.96, 950.25761812],
 [1609102800, 25772.55, 26444.65, 26184.83, 26388.94, 2844.36442243],
 [1609099200, 26134.79, 27070, 27058.39, 26194.55, 1839.56535711],
 [1609095600, 26951.01, 27274, 27024.2, 27058.39, 804.29465546],
 [1609092000, 26526, 27025, 26767.36, 27024.07, 1223.29475663],
 [1609088400, 26636.12, 27056.81, 27056.81, 26767.36, 1186.85176376],
 [1609084800, 26580, 27446.7, 27347.7, 27051.87, 2693.20303375],
 [1609081200, 27000, 27459.67, 27309.24, 27347.69, 1489.41402411],
 [1609077600, 27220.33, 27749.51, 27490.85, 27307.34, 1564.49125579],
 [1609074000, 27032.84, 27550, 27202.41, 27495.48, 1139.20603384],
 [1609070400, 26362.67, 27623.19, 27623.19, 27189.51, 2359.82878026],
 [1609066800, 26809.76, 28387, 27803.51, 27612.27, 2587.05855359],
 [1609063200, 27666.96, 27869.5, 27790, 27803.47, 38

In [188]:
def get_coinbase_data(start, end, granularity=86400, product_id="BTC-USD"):
    assert start >= datetime.datetime(2015, 7, 20)
    request_url = "https://api.pro.coinbase.com/products/{}/candles?start={}&end={}&granularity={}".format(product_id, start.isoformat(), end.isoformat(), granularity)
    r = requests.get(request_url)
    return r.json()

In [189]:
start = datetime.datetime(2015, 7, 20)
end = datetime.datetime.now()

def fetch_historic_data():
    curr_start = start
    curr_end = curr_start + datetime.timedelta(days=10)
    df = pd.DataFrame()
    while curr_start < end:
        data = get_coinbase_data(curr_start, curr_end, 3600)
        df = df.append(pd.DataFrame(np.array(data)[::-1]))
        
        curr_start = curr_end
        curr_end = curr_start + datetime.timedelta(days=10)
        time.sleep(0.4)
    return df

historic_df = fetch_historic_data()
historic_df.columns = ["time", "low", "high", "open", "close", "volume"]

In [190]:
historic_df = historic_df.drop_duplicates(subset=['time'])
historic_df.to_csv("../data/raw/bitcoin_prices_v3.csv")
pd.read_csv("../data/raw/bitcoin_prices_v3.csv")

,Unnamed: 0,time,low,high,open,close,volume
0,0,1.437426e+09,277.92,278.00,277.98,278.00,59.260456
1,1,1.437430e+09,277.37,278.50,278.00,278.14,346.995411
2,2,1.437433e+09,278.15,280.00,278.31,280.00,376.627552
3,3,1.437437e+09,279.38,281.00,279.96,280.89,288.177976
4,4,1.437440e+09,280.35,281.15,280.89,280.36,380.933023
...,...,...,...,...,...,...,...
47664,207,1.609168e+09,26775.94,27295.67,27251.85,26883.52,1587.736538
47665,208,1.609171e+09,26708.43,27141.45,26880.90,27030.27,1018.701436
47666,209,1.609175e+09,27002.76,27200.00,27030.27,27114.27,792.641173
47667,210,1.609178e+09,26880.93,27200.00,27115.67,26982.81,862.907883


### Fill in missing data values with linear interpolation

In [191]:
## Clean data and save as csv

df = pd.read_csv("../data/raw/bitcoin_prices_v1.csv")

start_time = df.iloc[0]["time"]
end_time = df.iloc[-1]["time"]
print(start_time, end_time)
print(df.shape)

all_data = pd.DataFrame(np.arange(start_time,end_time+1,3600))
all_data.columns = ["time"]

all_data = all_data.merge(df, 'left')

all_data = all_data.interpolate() # fill in missing data

all_data.columns = ["Time","","Low","High","Open","Close","Volume"]

all_data = all_data.drop([""], axis=1)
all_data.info()


1437426000.0 1609135200.0
(47656, 7)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47698 entries, 0 to 47697
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    47698 non-null  float64
 1   Low     47698 non-null  float64
 2   High    47698 non-null  float64
 3   Open    47698 non-null  float64
 4   Close   47698 non-null  float64
 5   Volume  47698 non-null  float64
dtypes: float64(6)
memory usage: 2.5 MB


### Clean up clear outliers/bad data points
Code snippet below shows that row 15242 has an open and low of 0.06 despite the high and close being well over $1000. Solution is to remove bad values and interpolate

In [192]:
open_mins = all_data.iloc[np.argsort(all_data["Open"])][:10]
open_maxes = all_data.iloc[np.argsort(all_data["Open"])][-10:]
print(open_mins.head())
print(open_maxes.head())

               Time     Low     High    Open    Close       Volume
15242  1.492297e+09    0.06  1186.60    0.06  1178.85   489.405359
844    1.440464e+09  200.00   207.38  200.00   206.81  2737.774928
847    1.440475e+09  203.78   210.84  205.50   208.54  1070.619850
845    1.440468e+09  206.05   210.00  207.10   210.00  1061.798656
848    1.440479e+09  208.33   214.49  208.45   211.02   650.274509
               Time       Low      High      Open     Close       Volume
47695  1.609128e+09  26820.00  27297.00  27265.66  26846.52   825.431767
47682  1.609081e+09  27000.00  27459.67  27309.24  27347.69  1489.414024
47683  1.609085e+09  26580.00  27446.70  27347.70  27051.87  2693.203034
47676  1.609060e+09  27201.23  27825.02  27445.37  27792.72   927.877877
47681  1.609078e+09  27220.33  27749.51  27490.85  27307.34  1564.491256


In [194]:
all_data["Low"].iloc[15242] = np.nan
all_data["Open"].iloc[15242] = np.nan
all_data = all_data.interpolate() # interpolate two missing values
all_data.describe()

,Time,Low,High,Open,Close,Volume
count,4.769800e+04,47698.000000,47698.000000,47698.000000,47698.000000,47698.000000
mean,1.523281e+09,5548.293815,5609.228933,5580.260680,5580.870338,517.242494
std,4.956974e+07,4651.227386,4711.102742,4682.634978,4683.569402,750.515535
min,1.437426e+09,198.020000,207.380000,200.000000,200.000000,0.280000
25%,1.480353e+09,748.455000,753.570000,751.112500,751.595000,182.558806
50%,1.523281e+09,5549.615000,5624.730000,5586.930000,5586.940000,312.708277
75%,1.566208e+09,8852.420000,8948.000000,8902.497500,8902.917500,577.812748
max,1.609135e+09,27666.960000,28387.000000,27803.510000,27803.470000,31505.461253


In [195]:
all_data.to_csv("../data/clean/bitcoin_prices.csv")